In [1]:
import os
import polars as pl

In [2]:
pl.Config(tbl_rows=100)

In [3]:
df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet")).rename({'autorstvo_kod':'100_7'})
df = pl.concat([df, pl.read_parquet(os.path.join("data","cnb_ceska_poezie.parquet"))])
spisovatelstvo = list(set(df.select(pl.col('100_7')).drop_nulls().to_series().to_list()))

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [5]:
df = df.filter(pl.col("100_7").is_in(spisovatelstvo))

In [6]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet")).explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(["100_7","374_a","375_a","678_a"])

In [7]:
aut

100_7,374_a,375_a,678_a
str,list[str],list[str],list[str]
"""jk01010001""",null,null,"[""Narozen 14.12.1931 v Osvračíně. MUDr., odborný asistent na klinice dětské neurologie fakulty dětského lékařství University Karlovy v Praze, práce z oboru.""]"
"""jk01010002""",null,null,"[""Narozen 19.11.1890 v Osvračíně u Horšova Týna, zemřel 21.7.1933 tamtéž. Ředitel školy v Klenčí, odborný učitel v Domažlicích, básník.""]"
"""jk01010003""",null,null,"[""Narozen roku 1957. Student filosofické fakulty university v Brně, básně v časopisech.""]"
"""jk01010004""","[""katoličtí kněží"", ""básníci""]","[""muž""]","[""Narozen 27. 1. 1889 v Odrlicích na Moravě, zemřel 22. 7. 1963. Kněz v Milovanech u Čermné, básník.""]"
"""jk01010005""",null,null,"[""Narozen 16.6.1877 v Boskovicích, zemřel 6.10.1960 v Brně. PhDr., profesor geografie, antropolog, archeolog, speleolog, publikace v oboru.""]"
"""jk01010009""",null,null,"[""Narozen roku 1843 ve Fryštáku, zemřel 22.7.1882 v Boskovicích. MUDr., zámecký a obvodní lékař v Boskovicích, časopisecké články.""]"
"""jk01010010""",null,null,"[""Narozena 1.10.1904 v Olšance na Volyňsku - Ukrajina, zemřela v roce 1995. Úřednice Ústřední rady odborů, překladatelka z ruštiny.""]"
"""jk01010012""",null,null,"[""Narozen 8.3.1803 v Dobřanech, zemřel 3.12.1875 v Litoměřicích. Děkan kapituly v Litoměřicích, autor modlitební knihy, překladatel do dánštiny.""]"
"""jk01010013""",null,null,"[""Narozen roku 1832, zemřel roku 1880. Povídkář v Poličce, příspěvky do časopisů.""]"


In [8]:
df = df.join(aut, left_on="100_7", right_on="100_7", how="left")

In [9]:
df

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,374_a,375_a,678_a
str,str,str,list[str],str,str,list[str],str,str,str,list[str],list[str],list[str]
"""1""","""Holub, Ota,""","""jk01041820""","[""aut""]","""1930-1992""",null,null,null,null,"""ck8300092""","[""publicisté"", ""spisovatelé""]","[""muž""]","[""Narozen roku 1930 v Malých Svatoňovicích, zemřel roku 1992. Publicista, žurnalista, vojenský historik. Specializoval se na československé vojenské dějiny a byl průkopníkem popularizace a zpřístupnění prvorepublikových bunkrů a opevnění.""]"
"""1""","""Tupý, Karel,""","""jk01140425""","[""aut""]","""1920-""",null,null,null,null,"""ck8300135""",null,null,"[""Narozen 20.5.1920 v Praze. PhDr., pedagog, práce z oboru.""]"
"""1""","""Smutný, Oldřich,""","""jk01120196""","[""aut"", ""pht""]","""1925-2013""",null,null,null,null,"""ck8300198""",null,null,"[""Narozen 17. 6. 1925 v Debři nad Jizerou, zemřel 1. 9. 2013 v Putimi u Písku. Pedagog, spisovatel, scénograf, kostýmní výtvarník, malíř.""]"
"""1""","""Tošner, Jindřich,""","""jn19990209929""","[""aut""]","""1947-""",null,null,null,null,"""ck8300221""","[""lékaři"", ""gynekologové"", … ""vysokoškolští učitelé""]","[""muž""]","[""Narozen 16. 10. 1947 v Žatci. Lékař, gynekolog, přednosta porodnicko-gynekologické kliniky LF UK a FN Hradec Králové (1991-2012). Majitel nakladatelství Medexart, vydavatel časopisu Gynekolog, básník, prozaik a publicista. Spoluautor učebnic gynekologie a porodnictví.""]"
"""1""","""Hons, Josef,""","""jk01041936""","[""aut""]","""1907-2001""",null,null,null,null,"""ck8300248""","[""spisovatelé"", ""básníci"", … ""vysokoškolští učitelé""]","[""muž""]","[""Narozen 16. 10. 1907 v Kutné Hoře, zemřel 26. 2. 2001 v Praze. Ing., vysokoškolský pedagog, prozaik píšící zvláště pro děti a mládež. Básník, publicista, autor odborných prací o dopravě.""]"
"""1""","""Škodová, Helena,""","""jx20040618039""","[""aut""]","""1955-""",null,null,null,null,"""ck8300304""",null,null,"[""Narozena 1955. Spisovatelka knih pro děti, populárně-naučné knihy.""]"
"""1""","""Krijtová, Olga,""","""jk01063330""","[""aut""]","""1931-2013""",null,null,null,null,"""ck8300334""",null,null,"[""Narozena 30.3.1931 v Hradci Králové, zemřela 7. 11. 2013. PhDr., nederlandistka, překladatelka zejména z holandštiny, též z angličtiny, knížka pro děti.""]"
"""1""","""Rzounek, Vítězslav,""","""jk01103438""","[""aut""]","""1921-2001""",null,null,null,null,"""ck8300395""",null,null,"[""Narozen 2.5.1921 ve Vlašimi, zemřel 11.2.2001. PhDr., DrSc., profesor české literatury, práce z oboru.""]"
"""1""","""Cincibuch, Petr,""","""jk01020728""","[""aut""]","""1943-""",null,null,null,null,"""ck8300396""",null,null,"[""Narozen 28. 6. 1943 v Praze. Básník, prozaik, redaktor kulturních časopisů, editor knih o historii automobilismu.""]"


In [10]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")

In [11]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")

In [12]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")

In [13]:
df

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,374_a,375_a,678_a,008,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z
str,str,str,list[str],str,str,list[str],str,str,str,list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""1""","""Holub, Ota,""","""jk01041820""","[""aut""]","""1930-1992""",null,null,null,null,"""ck8300092""","[""publicisté"", ""spisovatelé""]","[""muž""]","[""Narozen roku 1930 v Malých Svatoňovicích, zemřel roku 1992. Publicista, žurnalista, vojenský historik. Specializoval se na československé vojenské dějiny a byl průkopníkem popularizace a zpřístupnění prvorepublikových bunkrů a opevnění.""]","""830218s1982 xr a u0…","""1""","""0""","""Zrazené pevnosti /""",null,"""Ota Holub""",null,null,null,null,null,null,null,null,null,null,null,null
"""1""","""Tupý, Karel,""","""jk01140425""","[""aut""]","""1920-""",null,null,null,null,"""ck8300135""",null,null,"[""Narozen 20.5.1920 v Praze. PhDr., pedagog, práce z oboru.""]","""830323s1983 xr u0…","""1""","""0""","""Metodická příručka k Českému j…",null,"""Karel Tupý ; Bohumila Gregorov…",null,null,null,null,null,"[""7""]","[""metodické příručky""]","[""fd132826""]","[""czenas""]",null,null,null
"""1""","""Smutný, Oldřich,""","""jk01120196""","[""aut"", ""pht""]","""1925-2013""",null,null,null,null,"""ck8300198""",null,null,"[""Narozen 17. 6. 1925 v Debři nad Jizerou, zemřel 1. 9. 2013 v Putimi u Písku. Pedagog, spisovatel, scénograf, kostýmní výtvarník, malíř.""]","""951020s1982 xr af b 0…","""1""","""0""","""Labutě... labutě /""",null,"""Oldřich Smutný""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","[""vyprávění"", ""fotografické publikace"", … ""children's literature""]","[""fd133823"", ""fd132276"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null
"""1""","""Tošner, Jindřich,""","""jn19990209929""","[""aut""]","""1947-""",null,null,null,null,"""ck8300221""","[""lékaři"", ""gynekologové"", … ""vysokoškolští učitelé""]","[""muž""]","[""Narozen 16. 10. 1947 v Žatci. Lékař, gynekolog, přednosta porodnicko-gynekologické kliniky LF UK a FN Hradec Králové (1991-2012). Majitel nakladatelství Medexart, vydavatel časopisu Gynekolog, básník, prozaik a publicista. Spoluautor učebnic gynekologie a porodnictví.""]","""830323s1982 xr u0…","""1""","""0""","""Praktická cvičení z gynekologi…","""skriptum pro posl. lék. fakult…","""Jindřich Tošner ; Miroslav Cha…",null,null,null,null,null,"[""7""]","[""učebnice vysokých škol""]","[""fd133772""]","[""czenas""]",null,null,null
"""1""","""Hons, Josef,""","""jk01041936""","[""aut""]","""1907-2001""",null,null,null,null,"""ck8300248""","[""spisovatelé"", ""básníci"", … ""vysokoškolští učitelé""]","[""muž""]","[""Narozen 16. 10. 1907 v Kutné Hoře, zemřel 26. 2. 2001 v Praze. Ing., vysokoškolský pedagog, prozaik píšící zvláště pro děti a mládež. Básník, publicista, autor odborných prací o dopravě.""]","""830407s1983 xr a u0…","""1""","""0""","""Stavíme svět /""",null,"""Josef Hons ; il. Michal Brix""",null,null,null,null,null,"[""7""]","[""publikace pro děti""]","[""fd133156""]","[""czenas""]",null,null,null
"""1""","""Škodová, Helena,""","""jx20040618039""","[""aut""]","""1955-""",null,null,null,null,"""ck8300304""",null,null,"[""Narozena 1955. Spisovatelka knih pro děti, populárně-naučné knihy.""]","""830401s1982 xr a 0…","""1""","""0""","""Zákulisí velkoměsta :""","""[pro čtenáře od 8 let] /""","""Helena a Eduard Škodovi ; ilus…",null,null,null,null,null,"[""7""]","[""publikace pro děti""]","[""fd133156""]","[""czenas""]",null,null,null
"""1""","""Krijtová, Olga,""","""jk01063330""","[""aut""]","""1931-2013""",null,null,null,null,"""ck8300334""",null,null,"[""Narozena 30.3.1931 v Hradci Králové, zemřela 7. 11. 2013. PhDr., nederlandistka, překladatelka zejména z holandštiny, též z angličtiny, knížka pro děti.""]","""830303s1982 xr u

In [14]:
df.explode("655_a").group_by("655_a").len().top_k(100, by="len")

655_a,len
str,u32
null,61423
"""publikace pro děti""",19698
"""česká poezie""",17143
"""české romány""",16997
"""children's literature""",14162
"""Czech fiction""",12585
"""Czech poetry""",12197
"""české příběhy""",9660
"""české povídky""",9036


In [15]:
df = df.explode("655_a")

## 1. verze

In [17]:
poezie = df.filter(pl.col('655_a').str.contains('poez|poetr')).with_columns(kategorie = pl.lit('poezie'))
proza = df.filter(pl.col('655_a').str.contains('román|fiction|novel|příbě|povíd|stor|próz')).with_columns(kategorie = pl.lit('próza'))
vzpominky = df.filter(pl.col('655_a').str.contains('vzpomínky|autobiographical')).with_columns(kategorie = pl.lit('vzpomínky'))

In [18]:
do_grafu = pl.concat([poezie, proza, vzpominky])
from src.najdi_rok import najdi_rok
do_grafu = do_grafu.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
do_grafu = do_grafu.sort(by="rok").unique(subset=['100_a','245_a'],keep='first')

In [19]:
def narozeni(udaje):
    try:
        return int(udaje.split("-")[0])
    except:
        return None
        
def umrti(udaje):
    try:
        return int(udaje.split("-")[1])
    except:
        return None

In [20]:
do_grafu = do_grafu.with_columns(pl.col("100_d").map_elements(narozeni).alias('narozeni')).with_columns(pl.col("100_d").map_elements(umrti).alias('umrti'))

C:\Users\micha\AppData\Local\Temp\ipykernel_6076\2429155773.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  do_grafu = do_grafu.with_columns(pl.col("100_d").map_elements(narozeni).alias('narozeni')).with_columns(pl.col("100_d").map_elements(umrti).alias('umrti'))
C:\Users\micha\AppData\Local\Temp\ipykernel_6076\2429155773.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  do_grafu = do_grafu.with_columns(pl.col("100_d").map_elements(narozeni).alias('narozeni')).with_columns(pl.col("100_d").map_elements(umrti).alias('umrti'))


In [21]:
do_grafu = do_grafu.drop_nulls("narozeni")

In [22]:
do_grafu = do_grafu.with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [23]:
do_grafu = do_grafu.filter(pl.col("vek") > 6)

In [24]:
do_grafu.sort(by="vek").head(n=10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,374_a,375_a,678_a,008,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,kategorie,rok,narozeni,umrti,vek
str,str,str,list[str],str,str,list[str],str,str,str,list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],str,str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,i64,i64,i64,i64
"""1""","""Prager, Josef,""","""jk01100877""","[""aut""]","""1893-""",null,null,null,null,"""bkneli06814""",null,null,"[""Narozen 1.9.1893 v Praze. Strojník, prozaik, dramatik.""]","""020810s1900 xr f |…","""1""","""0""","""Vyvrženci :""","""Román /""","""Josef Prager""",null,null,null,null,null,"[""7""]","""české romány""","[""fd133974""]","[""czenas""]",null,null,null,"""próza""",1900,1893,null,7
"""1""","""Bílá, Viola Zoe,""","""jo2017972267""","[""aut"", ""ill""]","""2009-""",null,null,null,null,"""nkc20172895241""",null,"[""žena""]","[""Narozena 8. 12. 2009. Autorka ilustrací a písniček pro děti.""]","""171115t20172017xr ag a 0…","""1""","""0""","""Jak pejsek a kočička pěstovali…",null,"""Viola Zoe Bílá, Jonáš Bílý, Al…",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2017,2009,null,8
"""1""","""Sobotková, Anna,""","""mzk2008479487""","[""ill""]","""1998-""",null,null,null,null,"""nkc20081823565""",null,null,"[""Narozena 1998. Autorka ilustrací pohádek a příběhů pro děti.""]","""081110s2008 xr a c 0…","""1""","""0""","""Chaloupka mezi lesy /""",null,"""[ilustrace] Anna Sobotková & […",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2008,1998,null,10
"""1""","""Šmerda, Jakub,""","""mzk2005309121""","[""aut"", ""ill""]","""1995-""",null,null,null,null,"""nkc20051580412""",null,null,"[""Narozen 1995 v Lysicích. Povídky dětského autora s vlastními ilustracemi knihy.""]","""050831s2005 xr a c 0…","""1""","""0""","""Povídáme si s přírodou /""",null,"""Jakub Šmerda""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2005,1995,null,10
"""1""","""Bašová, Kateřina,""","""mzk2012695337""","[""aut"", ""ill""]","""2001-""",null,null,null,null,"""nkc20122359263""",null,null,"[""Narozena 2011 v Rakovníku. Autorka a ilustrátorka publikací pro děti.""]","""120405s2012 xr a b 0…","""1""","""0""","""Nelly na ostrově skřítků a elf…",null,"""Kateřina Bašová""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2012,2001,null,11
"""1""","""Fialová, Nela,""","""xx0195899""","[""aut"", ""ill""]","""2004-""",null,null,null,null,"""nkc20152722712""",null,"[""žena""]","[""Narozena 31. 5. 2004. Autorka poezie.""]","""150713t20152015xr a b 0…","""1""","""0""","""Slepice v Africe /""",null,"""Nela Fialová ; ilustrace v tex…",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""česká poezie""","[""fd133958"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""poezie""",2015,2004,null,11
"""1""","""Bradáčová, Sofie,""","""mzk2018979890""","[""aut""]","""2006-""",null,null,null,null,"""nkc20182970541""","[""spisovatelky""]","[""žena""]","[""Narozena 21. 4. 2006. Autorka knihy pro děti.""]","""180117s2017 xr a b 0…","""1""","""0""","""Kouzelná kniha /""",null,"""Sofie Bradáčová ; ilustrovala …",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd133156"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2017,2006,null,11
"""1""","""Horák, Radoslav,""","""ola2006340113""","[""aut""]","""1995-""",null,nul

In [25]:
do_grafu = do_grafu.filter((pl.col("umrti") >= pl.col('rok')) | pl.col("umrti").is_null())

In [26]:
do_grafu = do_grafu.filter(pl.col("vek") < 110)

In [27]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [28]:
do_grafu

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,374_a,375_a,678_a,008,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,kategorie,rok,narozeni,umrti,vek
str,str,str,list[str],str,str,list[str],str,str,str,list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],str,str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,i64,i64,i64,i64
"""1""","""Kraus, Ivan,""","""jn19981001677""","[""aut""]","""1939-""",null,null,null,null,"""nkc20162825206""",null,null,"[""Narozen 1.3.1939 v Praze. Herec, scénárista, loutkář, spisovatel, fejetonista, povídkář, emigrant po 1968.""]","""160829s2016 xr g 0…","""1""","""0""","""Ve vlastních názorech se shodn…",null,"""Ivan Kraus""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české prózy""","[""fd133972"", ""fd131784"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2016,1939,null,77
"""1""","""Pohořelý, Josef Mírumil,""","""jk01100313""","[""aut""]","""1805-1887""",null,null,null,null,"""nkc20122348407""",null,null,"[""Narozen 15. 3. 1805 v Kutné Hoře, zemřel 27. 3. 1887 tamtéž. Katolický kněz, prozaik, náboženské a didaktické spisy, překlady z němčiny.""]","""120319s1831 xr j 0…","""1""","""0""","""Eduard und Christine :""","""eine erbauliche Erzählung /""","""vom Joseph Pohořelý ; übersetz…",null,null,null,null,null,"[""7"", ""7"", ""7""]","""české prózy""","[""fd133972"", ""fd132016"", ""fd133157""]","[""czenas"", ""czenas"", ""czenas""]",null,null,null,"""próza""",1831,1805,1887,26
"""1""","""Šťastný, Vladimír,""","""jk01131144""","[""aut""]","""1841-1910""",null,null,null,null,"""nkc20071729754""","[""kněží"", ""spisovatelé"", ""dramatici""]","[""muž""]","[""Narozen 17. 3. 1841 v Rudíkově, zemřel 20. 8. 1910 v Obřanech. Kněz, básník, prozaik, dramatik, vydavatel a organizátor katolické literatury na Moravě, náboženské práce.""]","""070730s1892 xr e 0…","""1""","""0""","""Hlasy a ohlasy :""","""básně Vladimíra Šťastného""",null,null,null,null,null,null,"[""7""]","""česká poezie""","[""fd133958""]","[""czenas""]",null,null,null,"""poezie""",1892,1841,1910,51
"""1""","""Picko-Zásmucký, František,""","""jk01092987""","[""aut""]","""1868-1931""",null,null,null,null,"""np9439819""",null,null,"[""Narozen 23.9.1868 v Praze, zemřel 20.5.1931 v Mělníku. Beletrista -autor povídek, humoresek a kupletů, překladatel z němčiny a francouzštiny - hlavně dramata, publikoval též v okruhu Švába Malostranského.""]","""950403s1927 xr g u0…","""1""","""0""","""Když život zabouří a jiné poví…",null,"""napsal František Picko Zásmuck…",null,null,null,null,null,"[""7""]","""české povídky""","[""fd133971""]","[""czenas""]",null,null,null,"""próza""",1927,1868,1931,59
"""1""","""Klostermann, Karel,""","""jk01060677""","[""aut""]","""1848-1923""",null,null,null,null,"""nkc20081800644""","[""spisovatelé"", ""novináři"", ""středoškolští učitelé""]","[""muž""]","[""Narozen 13. 2. 1848 v Haagu (Rakousko), zemřel 16. 7. 1923 ve Štěkni u Strakonic. Český spisovatel narozený německým rodičům. Novinář, středoškolský profesor v Plzni. Beletrista.""]","""080604s1922 xr e 0…","""1""","""0""","""Vypovězen :""","""obraz ze života jihočeských dr…","""Karel Klostermann""",null,null,null,null,null,"[""7""]","""české romány""","[""fd133974""]","[""czenas""]",null,null,null,"""próza""",1922,1848,1923,74
"""1""","""Fořtel, Josef,""","""jk01031633""","[""aut""]","""1890-1977""",null,null,null,null,"""nos190110838""",null,null,"[""Narozen 4.2.1890 ve Spáleném Poříčí, zemřel 18.4.1977 ve Šlapánově u Benešova. Katolický kněz, literární recenzent, romanopisec, redaktor kalendářů.""]","""000918s1923 xr g 0…","""1""","""0""","""Odpadlík :""","""román z doby současné /""","""napsal Josef Fořtel""",null,null,null,null,null,"[""7""]","""české romány""","[""fd133974""]","[""czenas""]",null,null,null,"""próza""",1923,1890,1977,33
"""1""","""Tonder, Čeněk,""","""jk01132762""","[""trl""]","""1859-19

In [29]:
do_grafu2 = do_grafu.filter(pl.col("rok") < 1900).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [30]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1875,1925)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1925,1975)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})
print(len(do_grafu.filter(pl.col("rok").is_between(1925,1975))))

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

5047


C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [32]:
do_grafu2 = do_grafu.filter(pl.col("rok").is_between(1995,2025)).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})
print(len(do_grafu.filter(pl.col("rok").is_between(1995,2025))))

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

34557


C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [33]:
do_grafu2 = do_grafu.filter(pl.col("rok") > 1950).group_by(["vek","kategorie"]).len().rename({"len":"počet","vek":"věk"})

alt.Chart(do_grafu2.to_pandas()).mark_line().encode(
    alt.X('věk:Q').axis(domain=False, tickSize=0),
    alt.Y('počet:Q').stack('zero').axis(None),
    alt.Color('kategorie:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Mediánový věk

In [35]:
do_grafu.group_by("kategorie").agg(pl.col("vek").median())

kategorie,vek
str,f64
"""poezie""",52.0
"""vzpomínky""",66.0
"""próza""",51.0


## Druhý pokus

In [37]:
do_grafu.sample(10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,374_a,375_a,678_a,008,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,kategorie,rok,narozeni,umrti,vek
str,str,str,list[str],str,str,list[str],str,str,str,list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],str,str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,i64,i64,i64,i64
"""1""","""Kovanda, Jaroslav,""","""jk01062422""","[""aut""]","""1941-""",null,null,null,null,"""nkc20243590273""","[""výtvarní umělci"", ""sochaři"", … ""publicisté""]","[""muž""]","[""Narozen 26. 2. 1941 ve Zlíně. Výtvarník, sochař, malíř, typograf, básník a prozaik, publicista, vydavatel poetického magazínu, regionální literatura Zlínska.""]","""240220s2023 xr a g 0…","""1""","""0""","""Příběh opravdové lokomotivy /""",null,"""Jaroslav Kovanda ; ilustrace M…",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""česká poezie""","[""fd133958"", ""fd131893"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""poezie""",2023,1941,null,82
"""1""","""Drahokoupil, Filip,""","""mzk2009521535""","[""aut""]","""1986-""",null,null,null,null,"""nkc20243634754""",null,null,"[""Narozen 1986. Autor scénářů, povídek, fantasy literatury, též autor písňových textů a ilustrací vlastní literatury.""]","""241007s2024 xr a g 0…","""1""","""0""","""Volání bludiček /""",null,"""Drahokoupil Filip""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd184198"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2024,1986,null,38
"""1""","""Vrchlický, Jaroslav,""","""jk01151037""","[""aut""]","""1853-1912""",null,null,null,null,"""cpk20011029136""","[""spisovatelé"", ""básníci"", ""překladatelé""]","[""muž""]","[""Narozen 17. 2. 1853 v Lounech, zemřel 9. 9. 1912 v Domažlicích. Básník, prozaik, dramatik, překladatel, zejména z románských literatur. Profesor srovnávací literatury.""]","""011101s1880 xr e 0…","""1""","""0""","""Eklogy a písně /""",null,"""básně Jaroslava Vrchlického""",null,null,null,null,null,"[""7""]","""česká poezie""","[""fd133958""]","[""czenas""]",null,null,null,"""poezie""",1880,1853,1912,27
"""1""","""Houdek, Jiří,""","""mzk2007394718""","[""aut""]","""1958-""",null,null,null,null,"""nkc20233493534""",null,null,"[""Narozen 8.5.1958 v Praze. Mgr., specializace filmová věda a scénáristika, novinář, redaktor společenskovědních periodik, autor životopisné monografie, detektivních příběhů, též televizní a rozhlasová publicistika a odborné překlady z ruštiny.""]","""230120s2023 xr g 0…","""1""","""0""","""Pokažená oslava /""",null,"""Jiří Houdek""",null,null,null,null,null,"[""7"", ""7"", … ""9""]","""české příběhy""","[""fd133973"", ""fd132020"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"""próza""",2023,1958,null,65
"""0""","""Fringilla,""","""jk01031977""","[""aut""]","""1884-1955""",null,null,null,null,"""bk193400877""",null,null,"[""Narozena 31.7.1884 v Malíně u Kutné Hory, zemřela 19.8.1955 v Praze. Autorka dívčích románů.""]","""000211s1934 xr a g 0…","""1""","""0""","""Jožka vychovatelkou /""",null,"""Fringilla ; ilustrovala Ludmil…",null,null,null,null,null,"[""7"", ""7""]","""české romány""","[""fd133974"", ""fd132840""]","[""czenas"", ""czenas""]",null,null,null,"""próza""",1934,1884,1955,50
"""1""","""Schulz, Antonín,""","""jk01111143""","[""aut""]","""1852-1922""",null,null,null,null,"""bknhra09057""","[""spisovatelé"", ""dramatici"", ""archiváři""]","[""muž""]","[""Narozen 20. 2. 1852 v Pelhřimově, zemřel 16. 2. 1922 ve Dvoře Králové nad Labem. Archivář, historický spisovatel, dramatik, kulturní pracovník.""]","""041220s1914 xr e |…","""1""","""0""","""Zánik /""",null,"""Román od Antonína Schulze""",null,null,null,null,null,"[""7""]","""české romány""","[""fd133974""]","[""czenas""]",null,null,null,"""próza""",1914,1852,1922,62
"""1""","""Brabcová, Eva,""","""xx0000002""

In [38]:
do_grafu_drive = do_grafu.filter(pl.col("rok").is_between(1800,1938)).with_columns(obdobi = pl.lit('1800-1938'))
print(len(do_grafu_drive))
do_grafu_ted = do_grafu.filter(pl.col("rok").is_between(2000,2020)).with_columns(obdobi = pl.lit('2000-2020'))
print(len(do_grafu_ted))

7646
25431


In [39]:
do_grafu_basnici = pl.concat([do_grafu_drive.filter(pl.col("kategorie") == "poezie"), do_grafu_ted.filter(pl.col("kategorie") == "poezie")]).group_by(["obdobi","vek"]).len()
do_grafu_basnici

obdobi,vek,len
str,i64,u32
"""2000-2020""",85,61
"""1800-1938""",43,33
"""1800-1938""",63,25
"""1800-1938""",18,2
"""1800-1938""",68,20
"""2000-2020""",33,97
"""2000-2020""",47,92
"""2000-2020""",21,38
"""1800-1938""",32,57


In [40]:
def irozhlas_conf():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 14,
                      'anchor': 'start',
                       'fontWeight': 'bolder',
                    'subtitleFont': 'Noticia Text',
                       'subtitleLineHeight': 18,
                      'subtitleFontSize': 12,
                       'subtitleFontWeight': 'normal',
                      'subtitlePadding': 10,
                       'dy': -12,
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                 "fontWeight": "lighter",
                "titleFontWeight": "lighter",
                "labelFontSize": 10,
                "titleFontSize": 10,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 10,
                "titleFontSize": 10
            }
        }
    }

alt.themes.register('irozhlas', irozhlas_conf)
alt.themes.enable('irozhlas')

ThemeRegistry.enable('irozhlas')

In [41]:
tvar1 = alt.Chart(do_grafu_basnici.to_pandas(), width=300, height=100).mark_line(interpolate="monotone").encode(
    alt.X('vek:Q').axis(domain=False, tickSize=0, title=None),
    alt.Y('len:Q').stack('zero').axis(None),
    alt.Color('obdobi:N', sort=['2000-2020','1800-1838'], scale=alt.Scale(range=['#D6534B', '#DB842F']), legend=alt.Legend(title=None, orient="top")),
    alt.StrokeDash('obdobi:N', sort=['2000-2020','1800-1838'], legend=None)
)

tvar1

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [42]:
do_grafu_prozaici = pl.concat([do_grafu_drive.filter(pl.col("kategorie") == "próza"), do_grafu_ted.filter(pl.col("kategorie") == "próza")]).group_by(["obdobi","vek"]).len()

In [43]:
tvar2 = alt.Chart(do_grafu_prozaici.to_pandas(), width=300, height=100, title="…beletrii…").mark_line(interpolate="monotone").encode(
    alt.X('vek:Q').axis(domain=False, tickSize=0, title=None),
    alt.Y('len:Q').stack('zero').axis(None),
    alt.Color('obdobi:N', sort=['2000-2020','1800-1838'], scale=alt.Scale(range=['#D6534B', '#DB842F']), legend=None),
    alt.StrokeDash('obdobi:N', sort=['2000-2020','1800-1838'], legend=None)
)

tvar2

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [44]:
do_grafu_vzpominky = pl.concat([do_grafu_drive.filter(pl.col("kategorie") == "vzpomínky"), do_grafu_ted.filter(pl.col("kategorie") == "vzpomínky")]).group_by(["obdobi","vek"]).len()

In [45]:
tvar3 = alt.Chart(do_grafu_vzpominky.to_pandas(), width=300, height=100, title="…a vzpomínky").mark_line(interpolate="monotone").encode(
    alt.X('vek:Q').axis(domain=False, tickSize=0, title=None),
    alt.Y('len:Q').stack('zero').axis(None),
    alt.Color('obdobi:N', sort=['2000-2020','1800-1838'], scale=alt.Scale(range=['#D6534B', '#DB842F']), legend=None),
    alt.StrokeDash('obdobi:N', sort=['2000-2020','1800-1838'], legend=None)
)

tvar3

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [46]:
alt.vconcat(tvar1, tvar2, tvar3, title=alt.Title("V jakém věku lidé vydávají básně…")).resolve_scale(x='shared').configure_view(stroke='transparent')

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)